# Introduction to Image query, download and display

**Base:** For Rubin DP1 in the RSP or externally

**LSST data products:** `visit_image`, `deep_coadd`, `template_coadd`, `difference_image`

**SSSC rsp_queries module:** `query_images`

**Packages used:** `lsst.rsp.utils`, `pyvo`, `lsst.rsp.service`, `astropy.time`, `astropy.table`, `astropy.coordinates`, `astropy.units`, [`lsst.afw.display`, `pyds9`]

**Authors:** Originally developed by Tim Lister


## 1. Introduction

This notebook demonstrates how to find images using the `ivoa.ObsCore`, retrieve images from a remote location using the [IVOA datalink](https://www.ivoa.net/documents/DataLink/20211115/WD-DataLink-1.1-20211115.html) protocol and download image cutouts using the [Server-side Operations for Data Access (SODA)](https://www.ivoa.net/documents/SODA/20170517/REC-SODA-1.0.html) protocal.

### 1.1 Import packages
Import the rsp_queries module and some common astronomy packages 

In [ ]:
import os
from datetime import datetime

from astropy.time import Time
from astropy import units as u
from astropy.coordinates import SkyCoord, CustomBarycentricEcliptic

from sso_query.query_images import build_query, make_query, download_data, get_image_cutout, display_image


### 1.2 Connecting to the RSP

Checking for access to the RSP and checking whether `ACCESS_TOKEN` and `EXTERNAL_INSTANCE_URL` have been set if running externally, is handled by `check_rsp_access()`. This is run by the query routines and doesn't need to be done by the user (although it can be;  `check_rsp_access()` will return `True` if access is good)

## 2. Find images

### 2.1 Query building
The first step is to find some images to work with. This is done by `build_query()` (which is called from the query runner `make_query()`). This routine is designed to handle the coordinates, filter bands and times in a (hopefully) SSSC user-friendly way.

We define the center right ascension and declination of our target field which is the low ecliptic latitude 'Rubin SV 38 7' field which has the majority of the Solar System Object (SSO) detections (5304), and unique objects (406).

In [ ]:
target_ra = 37.86
target_dec = +6.98
center = SkyCoord(target_ra, target_dec, unit='deg', frame='icrs')
print(f"RA/Dec= {center.to_string('hmsdms', sep=' ')}, ecliptic latitude={ center.transform_to(CustomBarycentricEcliptic).lat:.1f}")

For times, `make_query()` will take either Modified Julian Dates (MJD) in the TAI timescale as a `float` if you have those from e.g. another tutorial notebook or a `astropy.Time` object in any format and timescale.

(There is 37s difference between UTC and TAI in 2024, hence the difference from .0 above)

In [ ]:
time1 = Time(datetime(2024, 11, 27), scale='utc')
time2 = Time(datetime(2024, 12,  9), scale='utc')
print(f"Search Timespan MJD {time1.tai.mjd:.6f} -> {time2.tai.mjd:.6f} TAI")

Make and print the ADQL query string. By default the bands to be searched are the most relevant for SSO discovery, namely _g, r, i_ but this can be changed by doing e.g, `bands=['r', 'i', 'z', 'y']` in the call 

In [ ]:
query = build_query(center, t_min=time1, t_max=time2)
print(query)

To search for difference or coadded images, set `calib_level=3`:

In [ ]:
diffs_query = build_query(center, bands=['r', 'i', 'z', 'y'], t_min=60641.0, t_max=60653.0, calib_level=3)
print(diffs_query)

### 2.2 Query execution

`make_query()` takes the same options as `build_query()` (which it calls) and checks for access and actually runs the query and returns an `astropy.Table` of results.

In [ ]:
results = make_query(center, t_min=time1, t_max=time2)
print(results[('lsst_visit', 'lsst_band', 's_ra', 's_dec')])
## List of all the columns
print(results.colnames)

## 3. Data download

Once a table of results has been obtained, data can be downloaded by calling `download_data`. When first run, this will calculate the size of the downloaded data first; set `actually_download=True` to actually download to the chosen `output_directory` (which is created if necessary). To download the first 

In [ ]:
output_dir = os.path.join(os.path.expanduser('~'), 'Rubin_dl_data')
dl_files = download_data(results[0:2], output_dir)
print(dl_files)

Actually download the data


In [ ]:
dl_files = download_data(results[0:2], output_dir, actually_download=True)
print(dl_files)

In [ ]:
# Show files and size
status = os.system(f"ls -lh {output_dir}")

In [ ]:
# and tidy up
for file in dl_files:
    try:
        os.remove(file)
    except FileNotFoundError:
        pass
status = os.system(f"ls -lh {output_dir}")    

## 4. Image cutouts

We can cutout a region of the resulting images using `get_image_cutout()` which will return a `astropy.fits.HDUList` (see [Astropy FITS](https://docs.astropy.org/en/stable/io/fits/index.html) docs)

(There is currently no checking of whether the passed center is within the image boundaries; feel free to contribute one to the [repo](https://github.com/lsst-sssc/rsp_queries/)...)

We'll define a center for out cutout close to the reported center (`s_ra, s_dec`) of the first image of the results from above

In [ ]:
cutout_center = SkyCoord(37.76, 7.0, unit='deg')
hdulist = get_image_cutout(results[0], cutout_center, 20*u.arcsec)
print(hdulist.info())

In [ ]:
## Inside the RSP (or if you have the LSST Pipelines installed) this can display the image
#import lsst.afw.display as afwDisplay
#from lsst.afw.image import ExposureF
#afwDisplay.setDefaultBackend('firefly')
#afw_display = afwDisplay.Display(frame=1)
#afw_display.image(hdulist['IMAGE'])

## 4.1 Direct image download and display

`display_image()` can take a Datalink url from the `access_url` column of the results table and display it.
Let's display the last image in the results (this part should work inside RSP or outside (if you have DS9 installed))

In [ ]:
disp = display_image(results[-1]['access_url'])

In [ ]:
## Tidy up
del(hdulist, disp, results)